https://mlxai.github.io/2017/01/06/vectorized-implementation-of-svm-loss-and-gradient-update.html

In [127]:
import numpy as np
k = 3
d = 4000
n = 5
W_mat = np.random.randn(k ,d)
X_mat = np.random.randn(d ,n) * 0.5
b_vec = np.random.randn(k ,1)
y_vec = np.random.choice(k, n)
Y_mat = np.eye(k)[y_vec].T
lambda_ = 0.00


In [128]:

def compute_cost(X_mat, Y_mat, W_mat, b_vec, lambda_):
    """
    \sum_{i!=y}[max(0, \vec{s} - s_y + 1)]
    """
    n_data = X_mat.shape[1]
    s_mat = W_mat.dot(X_mat) + b_vec
    s_y_vec = np.sum(s_mat * Y_mat, axis=0) # shape = (ndata,)
    hinge_mat = np.maximum(s_mat - s_y_vec + 1, 0)

    # we summed n_data extra ones. Therefore,
    # (np.sum(hinge_mat) - n_data) / ndata
    ret = (np.sum(hinge_mat)/n_data - 1) + 0.5*lambda_*np.sum(W_mat**2)
    return ret

# def svm_loss_naive(W, X, y, reg):
#     num_classes = W.shape[1]
#     num_train = X.shape[0]
#     loss = 0.0

#     for i in range(num_train):
#         scores = X[i].dot(W)
#         correct_class_score = scores[y[i]]
#         for j in range(num_classes):
#             if j == y[i]:
#                 continue
#             margin = scores[j] - correct_class_score + 1 # note delta = 1
#             if margin > 0:
#                 loss += margin
#     loss /= num_train # mean
#     loss += 0.5 * reg * np.sum(W * W) # l2 regularization
#     return loss

def compute_gradients(X_mat, Y_mat, W_mat, b_vec, lambda_):
    n_data = X_mat.shape[1]
    #
    s_mat = W_mat.dot(X_mat) + b_vec
    s_y_vec = np.sum(s_mat * Y_mat, axis=0) # shape = (ndata,)

    # ------------------------
    # start making g_mat
    # ------------------------
    # hinge_mat = np.maximum(s_mat - s_y_vec + 1, 0)
    # g_mat = hinge_mat.copy()  # but we never use hinge_mat again
    g_mat = np.maximum(s_mat - s_y_vec + 1, 0)
    # for non correct classes
    g_mat[g_mat > 0] = 1
    # for the correct classes
    g_mat[Y_mat == 1] = 0
    # compute the gradient of the correct classes
    crt_cls_grad = -np.sum(g_mat, axis=0)
    g_mat += Y_mat * crt_cls_grad

    # build the returns
    grad_b = np.mean(g_mat, axis=1)[:, np.newaxis]
    grad_W = g_mat.dot(X_mat.T) / n_data
    grad_W += lambda_ * W_mat
    return (grad_W, grad_b)

def compute_grad_fwd_diff(X_mat, Y_mat, W_mat, b_vec, lambda_):
    """
    Translated from matlab version of ComputeGradsNum
    """
    h = 1e-6
    h_inv = 1.0 / h
    nclass = Y_mat.shape[0]
    ndim = X_mat.shape[0]
    grad_W = np.zeros(W_mat.shape)
    grad_b = np.zeros((nclass, 1))

    cost = compute_cost(X_mat, Y_mat, W_mat, b_vec, lambda_)

    for i in range(nclass):
        b_old = b_vec[i, 0]

        b_vec[i, 0] = b_old + h
        new_cost = compute_cost(X_mat, Y_mat, W_mat, b_vec, lambda_)
        grad_b[i, 0] = (new_cost - cost) * h_inv

        b_vec[i, 0] = b_old

    for idx in np.ndindex(W_mat.shape):
        w_old = W_mat[idx]

        W_mat[idx] = w_old + h
        new_cost = compute_cost(X_mat, Y_mat, W_mat, b_vec, lambda_)
        grad_W[idx] = (new_cost - cost) * h_inv

        W_mat[idx] = w_old

    return (grad_W, grad_b)
def compute_grad_central_diff(X_mat, Y_mat, W_mat, b_vec, lambda_):
    """
    Translated from matlab version of ComputeGradsNum
    """
    h = 1e-6
    h_inv = 1.0 / h
    nclass = Y_mat.shape[0]
    ndim = X_mat.shape[0]
    grad_W = np.zeros(W_mat.shape)
    grad_b = np.zeros((nclass, 1))

    for i in range(nclass):
        b_old = b_vec[i, 0]

        b_vec[i, 0] = b_old + h
        c1 = compute_cost(X_mat, Y_mat, W_mat, b_vec, lambda_)

        b_vec[i, 0] = b_old - h
        c2 = compute_cost(X_mat, Y_mat, W_mat, b_vec, lambda_)

        grad_b[i, 0] = (c1 - c2) * h_inv * 0.5

        b_vec[i, 0] = b_old

    for idx in np.ndindex(W_mat.shape):
        w_old = W_mat[idx]

        W_mat[idx] = w_old + h
        c1 = compute_cost(X_mat, Y_mat, W_mat, b_vec, lambda_)

        W_mat[idx] = w_old - h
        c2 = compute_cost(X_mat, Y_mat, W_mat, b_vec, lambda_)

        grad_W[idx] = (c1 - c2) * h_inv * 0.5

        W_mat[idx] = w_old

    return (grad_W, grad_b)

In [129]:
gW1, gb1 = compute_gradients(X_mat, Y_mat, W_mat, b_vec, lambda_)

In [130]:
gW2, gb2 = compute_grad_fwd_diff(X_mat, Y_mat, W_mat, b_vec, lambda_)

In [131]:
gW3, gb3 = compute_grad_central_diff(X_mat, Y_mat, W_mat, b_vec, lambda_)

In [133]:
gb1 - gb3

array([[5.60862634e-10],
       [1.87012572e-09],
       [1.12172527e-09]])

In [132]:
gb1 - gb2

array([[2.33721947e-09],
       [3.64648256e-09],
       [4.67443895e-09]])

In [134]:
gW1 - gW2

array([[ 4.90691004e-09,  1.42817699e-09,  1.84259967e-09, ...,
         3.13003956e-10,  3.54404306e-09,  7.64684427e-10],
       [-1.58215613e-09,  2.94500527e-09,  2.93273632e-09, ...,
         2.02499217e-09,  3.50433313e-09,  4.86502727e-09],
       [ 3.78067344e-09,  2.73224510e-09,  5.88280504e-09, ...,
         1.21471755e-09,  3.60976485e-09,  5.02842937e-09]])

In [135]:
gW1 - gW3

array([[ 1.35419637e-09, -3.48179846e-10,  6.62428307e-11, ...,
         3.13003956e-10, -8.67061978e-12, -1.01167241e-09],
       [-3.35851297e-09,  1.16864843e-09,  1.15637948e-09, ...,
         2.48635335e-10, -4.83805496e-11,  1.31231359e-09],
       [ 2.00431660e-09,  9.55888257e-10,  5.53734520e-10, ...,
        -5.61639291e-10,  5.70511693e-11, -3.00641151e-10]])